In [1]:
import torch.utils.data as data
from PIL import Image
import os
import numpy as np
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import matplotlib.pyplot as plt
from PIL import Image

def check_dir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)

class Img_to_zero_center(object):
    def __int__(self):
        pass
    def __call__(self, t_img):
        '''
        :param img:tensor be 0-1
        :return:
        '''
        t_img=(t_img-0.5)*2
        return t_img

class Reverse_zero_center(object):
    def __init__(self):
        pass
    def __call__(self,t_img):
        t_img=t_img/2+0.5
        return t_img

In [3]:
class CACD(data.Dataset):
    def __init__(self, split='train', transforms=None, label_transforms=None):
        list_root = "/kaggle/input/utkpreprocess/lists"
        data_root = "/kaggle/input/utkpreprocess/UTKFaceCrop"
        if split == "train":
            self.list_path=os.path.join(list_root,"train.txt")
        else:
            self.list_path = os.path.join(list_root, "test.txt")
        self.images_labels=[]#path
        self.transform=transforms
        self.label_transform=label_transforms
        with open(self.list_path) as fr:
            lines=fr.readlines()
            for line in lines:
                line.strip()
                item=line.split()
                image_label=[]
                image_label.append(os.path.join(data_root,item[0]))
                try:
                    image_label.append(np.array(int(item[1])))
                except:
                  print(item[0])
                  print(item[1])
                self.images_labels.append(image_label)

    def __getitem__(self, idx):
        img_path,label=self.images_labels[idx]
        img=Image.open(img_path)

        if self.transform is not None:
            img=self.transform(img)
        if self.label_transform is None:
            label=torch.from_numpy(label)
        return img,label

    def __len__(self):
        return len(self.images_labels)


transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[1., 1., 1.]),
])

CACD_dataset=CACD("train", transforms, None)
train_loader = torch.utils.data.DataLoader(
    dataset=CACD_dataset,
    batch_size=32,
    shuffle=False
)
for idx,(img,label) in enumerate(train_loader):
    print(img.size())
    print(label.size())
    break

torch.Size([32, 3, 400, 400])
torch.Size([32])


In [4]:
import math
from torch.nn.parameter import Parameter
from torch.nn.functional import pad
from torch.nn.modules import Module
from torch.nn.modules.utils import _single, _pair, _triple

class _ConvNd(Module):

    def __init__(self, in_channels, out_channels, kernel_size, stride,
                 padding, dilation, transposed, output_padding, groups, bias):
        super(_ConvNd, self).__init__()
        if in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.transposed = transposed
        self.output_padding = output_padding
        self.groups = groups
        if transposed:
            self.weight = Parameter(torch.Tensor(
                in_channels, out_channels // groups, *kernel_size))
        else:
            self.weight = Parameter(torch.Tensor(
                out_channels, in_channels // groups, *kernel_size))
        if bias:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        n = self.in_channels
        for k in self.kernel_size:
            n *= k
        stdv = 1. / math.sqrt(n)
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def __repr__(self):
        s = ('{name}({in_channels}, {out_channels}, kernel_size={kernel_size}'
             ', stride={stride}')
        if self.padding != (0,) * len(self.padding):
            s += ', padding={padding}'
        if self.dilation != (1,) * len(self.dilation):
            s += ', dilation={dilation}'
        if self.output_padding != (0,) * len(self.output_padding):
            s += ', output_padding={output_padding}'
        if self.groups != 1:
            s += ', groups={groups}'
        if self.bias is None:
            s += ', bias=False'
        s += ')'
        return s.format(name=self.__class__.__name__, **self.__dict__)

class Conv2d(_ConvNd):

    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True):
        kernel_size = _pair(kernel_size)
        stride = _pair(stride)
        padding = _pair(padding)
        dilation = _pair(dilation)
        super(Conv2d, self).__init__(
            in_channels, out_channels, kernel_size, stride, padding, dilation,
            False, _pair(0), groups, bias)

    def forward(self, input):
        return _conv2d_same_padding(input, self.weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)

# custom conv2d, because pytorch don't have "padding='same'" option.
def _conv2d_same_padding(input, weight, bias=None, stride=(1,1), padding=1, dilation=(1,1), groups=1):

    input_rows = input.size(2)
    filter_rows = weight.size(2)
    effective_filter_size_rows = (filter_rows - 1) * dilation[0] + 1
    out_rows = (input_rows + stride[0] - 1) // stride[0]
    padding_needed = max(0, (out_rows - 1) * stride[0] + effective_filter_size_rows -
                  input_rows)
    padding_rows = max(0, (out_rows - 1) * stride[0] +
                        (filter_rows - 1) * dilation[0] + 1 - input_rows)
    rows_odd = (padding_rows % 2 != 0)
    padding_cols = max(0, (out_rows - 1) * stride[0] +
                        (filter_rows - 1) * dilation[0] + 1 - input_rows)
    cols_odd = (padding_rows % 2 != 0)

    if rows_odd or cols_odd:
        input = pad(input, [0, int(cols_odd), 0, int(rows_odd)])

    return F.conv2d(input, weight, bias, stride,
                  padding=(padding_rows // 2, padding_cols // 2),
                  dilation=dilation, groups=groups)

In [7]:
class AgeAlexNet(nn.Module):
    def __init__(self,pretrainded=False,modelpath=None):
        super(AgeAlexNet, self).__init__()
        assert pretrainded is False or modelpath is not None,"pretrain model need to be specified"
        self.features = nn.Sequential(
            Conv2d(3, 96, kernel_size=11, stride=4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LocalResponseNorm(2,2e-5,0.75),

            Conv2d(96, 256, kernel_size=5, stride=1,groups=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LocalResponseNorm(2, 2e-5, 0.75),

            Conv2d(256, 384, kernel_size=3, stride=1),
            nn.ReLU(inplace=True),

            Conv2d(384, 384, kernel_size=3,stride=1,groups=2),
            nn.ReLU(inplace=True),

            Conv2d(384, 256, kernel_size=3,stride=1,groups=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.age_classifier=nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 5),
        )
        if pretrainded is True:
            self.load_pretrained_params(modelpath)

        self.Conv3_feature_module=nn.Sequential()
        self.Conv4_feature_module=nn.Sequential()
        self.Conv5_feature_module=nn.Sequential()
        self.Pool5_feature_module=nn.Sequential()
        for x in range(10):
            self.Conv3_feature_module.add_module(str(x), self.features[x])
        for x in range(10,12):
            self.Conv4_feature_module.add_module(str(x),self.features[x])
        for x in range(12,14):
            self.Conv5_feature_module.add_module(str(x),self.features[x])
        for x in range(14,15):
            self.Pool5_feature_module.add_module(str(x),self.features[x])


    def forward(self, x):
        self.conv3_feature=self.Conv3_feature_module(x)
        self.conv4_feature=self.Conv4_feature_module(self.conv3_feature)
        self.conv5_feature=self.Conv5_feature_module(self.conv4_feature)
        pool5_feature=self.Pool5_feature_module(self.conv5_feature)
        self.pool5_feature=pool5_feature
        flattened = pool5_feature.view(pool5_feature.size(0), -1)
        age_logit = self.age_classifier(flattened)
        return age_logit

    def load_pretrained_params(self,path):
        # step1: load pretrained model
        pretrained_dict = torch.load(path)
        # step2: get model state_dict
        model_dict = self.state_dict()
        # step3: remove pretrained_dict params which is not in model_dict
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        # step4: update model_dict using pretrained_dict
        model_dict.update(pretrained_dict)
        # step5: update model using model_dict
        self.load_state_dict(model_dict)


class AgeClassify:
    def __init__(self, modelz = AgeAlexNet(pretrainded=False).cuda()):
        #step 1:define model
        self.model=modelz
        #step 2:define optimizer
        self.optim=torch.optim.Adam(self.model.parameters(),lr=1e-4,betas=(0.5, 0.999))
        #step 3:define loss
        self.criterion=nn.CrossEntropyLoss().cuda()

    def train(self,input,label):
        self.model.train()
        output=self.model(input)
        self.loss=self.criterion(output,label)

    def val(self,input):
        self.model.eval()
        output=F.softmax(self.model(input),dim=1).max(1)[1]
        return output

    def save_model(self,dir,filename):
        torch.save(self.model.state_dict(),os.path.join(dir,filename))


batch_size 512 20 epochs

In [6]:
from tqdm import tqdm
import os

#define
max_epoches = 20
batch_size = 512
save_interval = 20000
val_interval = 20000
saved_model_folder = '/kaggle/working/checkpoint'
check_dir(saved_model_folder)

transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((227, 227)),
    torchvision.transforms.ToTensor(),
    Img_to_zero_center()
])

train_dataset = CACD("train", transforms, None)
test_dataset = CACD("test", transforms, None)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)

model=AgeClassify()
optim=model.optim

for epoch in range(max_epoches):
    for train_idx, (img,label) in enumerate(train_loader):
        img=img.cuda()
        label=label.cuda()

        #train
        optim.zero_grad()
        model.train(img,label)
        loss=model.loss
        loss.backward()
        optim.step()

        print('step %d/%d, cls_loss = %.3f' % (train_idx, len(train_loader), loss))

        # save the parameters at the end of each save interval
        if train_idx*batch_size % save_interval == 0:
            model.save_model(dir=saved_model_folder,
                              filename='epoch_%d_iter_%d.pth'%(epoch, train_idx))
            print('checkpoint has been created!')

        #val step

        if train_idx % val_interval == 0:
            train_correct=0
            train_total=0
            with torch.no_grad():
                for val_img,val_label in tqdm(test_loader):
                    val_img=val_img.cuda()
                    val_label=val_label.cuda()
                    output=model.val(val_img)
                    train_correct += (output == val_label).sum()
                    train_total += val_img.size()[0]

            print('validate has been finished!')
            print('val_acc = %.3f' % (train_correct.cpu().numpy()/train_total))

step 0/41, cls_loss = 1.612
checkpoint has been created!


100%|██████████| 5/5 [00:17<00:00,  3.59s/it]


validate has been finished!
val_acc = 0.330
step 1/41, cls_loss = 1.604
step 2/41, cls_loss = 1.586
step 3/41, cls_loss = 1.578
step 4/41, cls_loss = 1.547
step 5/41, cls_loss = 1.554
step 6/41, cls_loss = 1.522
step 7/41, cls_loss = 1.564
step 8/41, cls_loss = 1.543
step 9/41, cls_loss = 1.524
step 10/41, cls_loss = 1.522
step 11/41, cls_loss = 1.501
step 12/41, cls_loss = 1.527
step 13/41, cls_loss = 1.537
step 14/41, cls_loss = 1.540
step 15/41, cls_loss = 1.486
step 16/41, cls_loss = 1.503
step 17/41, cls_loss = 1.537
step 18/41, cls_loss = 1.531
step 19/41, cls_loss = 1.530
step 20/41, cls_loss = 1.476
step 21/41, cls_loss = 1.542
step 22/41, cls_loss = 1.455
step 23/41, cls_loss = 1.524
step 24/41, cls_loss = 1.495
step 25/41, cls_loss = 1.505
step 26/41, cls_loss = 1.488
step 27/41, cls_loss = 1.507
step 28/41, cls_loss = 1.480
step 29/41, cls_loss = 1.471
step 30/41, cls_loss = 1.485
step 31/41, cls_loss = 1.500
step 32/41, cls_loss = 1.479
step 33/41, cls_loss = 1.449
step 34/

100%|██████████| 5/5 [00:10<00:00,  2.02s/it]


validate has been finished!
val_acc = 0.357
step 1/41, cls_loss = 1.493
step 2/41, cls_loss = 1.452
step 3/41, cls_loss = 1.397
step 4/41, cls_loss = 1.456
step 5/41, cls_loss = 1.526
step 6/41, cls_loss = 1.406
step 7/41, cls_loss = 1.468
step 8/41, cls_loss = 1.433
step 9/41, cls_loss = 1.402
step 10/41, cls_loss = 1.370
step 11/41, cls_loss = 1.421
step 12/41, cls_loss = 1.541
step 13/41, cls_loss = 1.485
step 14/41, cls_loss = 1.407
step 15/41, cls_loss = 1.521
step 16/41, cls_loss = 1.445
step 17/41, cls_loss = 1.419
step 18/41, cls_loss = 1.359
step 19/41, cls_loss = 1.434
step 20/41, cls_loss = 1.460
step 21/41, cls_loss = 1.418
step 22/41, cls_loss = 1.329
step 23/41, cls_loss = 1.341
step 24/41, cls_loss = 1.361
step 25/41, cls_loss = 1.389
step 26/41, cls_loss = 1.355
step 27/41, cls_loss = 1.295
step 28/41, cls_loss = 1.294
step 29/41, cls_loss = 1.295
step 30/41, cls_loss = 1.249
step 31/41, cls_loss = 1.318
step 32/41, cls_loss = 1.304
step 33/41, cls_loss = 1.292
step 34/

...

100%|██████████| 5/5 [00:11<00:00,  2.23s/it]


validate has been finished!
val_acc = 0.627
step 1/41, cls_loss = 0.759
step 2/41, cls_loss = 0.798
step 3/41, cls_loss = 0.626
step 4/41, cls_loss = 0.644
step 5/41, cls_loss = 0.627
step 6/41, cls_loss = 0.757
step 7/41, cls_loss = 0.789
step 8/41, cls_loss = 0.671
step 9/41, cls_loss = 0.661
step 10/41, cls_loss = 0.636
step 11/41, cls_loss = 0.693
step 12/41, cls_loss = 0.678
step 13/41, cls_loss = 0.624
step 14/41, cls_loss = 0.600
step 15/41, cls_loss = 0.614
step 16/41, cls_loss = 0.597
step 17/41, cls_loss = 0.690
step 18/41, cls_loss = 0.615
step 19/41, cls_loss = 0.751
step 20/41, cls_loss = 0.699
step 21/41, cls_loss = 0.661
step 22/41, cls_loss = 0.679
step 23/41, cls_loss = 0.705
step 24/41, cls_loss = 0.681
step 25/41, cls_loss = 0.651
step 26/41, cls_loss = 0.701
step 27/41, cls_loss = 0.642
step 28/41, cls_loss = 0.604
step 29/41, cls_loss = 0.704
step 30/41, cls_loss = 0.634
step 31/41, cls_loss = 0.609
step 32/41, cls_loss = 0.696
step 33/41, cls_loss = 0.636
step 34/

100%|██████████| 5/5 [00:10<00:00,  2.05s/it]


validate has been finished!
val_acc = 0.659
step 1/41, cls_loss = 0.618
step 2/41, cls_loss = 0.620
step 3/41, cls_loss = 0.676
step 4/41, cls_loss = 0.641


KeyboardInterrupt: 

batch_size 64 epoch 20

In [10]:
modelz = AgeAlexNet(pretrainded=True,modelpath="/kaggle/working/checkpoint/epoch_20_iter_0.pth").cuda()
model=AgeClassify(modelz)
optim=torch.optim.Adam(modelz.parameters(),lr=1e-5,betas=(0.5, 0.999))

<ipython-input-7-cc13c5de9fd8>:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_dict = torch.load(path)


In [15]:
from tqdm import tqdm
import os

#define
max_epoches = 10
batch_size = 64
save_interval = 20000
val_interval = 20000

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)

saved_model_folder = '/kaggle/working/checkpoint2'
check_dir(saved_model_folder)

for epoch in range(max_epoches):
    for train_idx, (img,label) in enumerate(train_loader):
        img=img.cuda()
        label=label.cuda()

        #train
        optim.zero_grad()
        model.train(img,label)
        loss=model.loss
        loss.backward()
        optim.step()

        if train_idx % 36 == 0:
            print('step %d/%d, cls_loss = %.3f' % (train_idx, len(train_loader), loss))

        # save the parameters at the end of each save interval
        if train_idx*batch_size % save_interval == 0:
            model.save_model(dir=saved_model_folder,
                              filename='epoch_%d_iter_%d.pth'%(epoch, train_idx))
            print('checkpoint has been created!')

        #val step

        if train_idx % val_interval == 0:
            train_correct=0
            train_total=0
            with torch.no_grad():
                for val_img,val_label in tqdm(test_loader):
                    val_img=val_img.cuda()
                    val_label=val_label.cuda()
                    output=model.val(val_img)
                    train_correct += (output == val_label).sum()
                    train_total += val_img.size()[0]

            print('validate has been finished!')
            print('val_acc = %.3f' % (train_correct.cpu().numpy()/train_total))

step 0/328, cls_loss = 0.770
checkpoint has been created!


100%|██████████| 36/36 [00:09<00:00,  3.85it/s]


validate has been finished!
val_acc = 0.666
step 36/328, cls_loss = 0.516
step 72/328, cls_loss = 0.651
step 108/328, cls_loss = 0.480
step 144/328, cls_loss = 0.545
step 180/328, cls_loss = 0.577
step 216/328, cls_loss = 0.615
step 252/328, cls_loss = 0.469
step 288/328, cls_loss = 0.461
step 324/328, cls_loss = 0.574
step 0/328, cls_loss = 0.630
checkpoint has been created!


100%|██████████| 36/36 [00:09<00:00,  3.73it/s]


validate has been finished!
val_acc = 0.652
step 36/328, cls_loss = 0.577
step 72/328, cls_loss = 0.428
step 108/328, cls_loss = 0.707
step 144/328, cls_loss = 0.448
step 180/328, cls_loss = 0.734
step 216/328, cls_loss = 0.561
step 252/328, cls_loss = 0.511
step 288/328, cls_loss = 0.483
step 324/328, cls_loss = 0.420
step 0/328, cls_loss = 0.555
checkpoint has been created!


...

validate has been finished!
val_acc = 0.652
step 36/328, cls_loss = 0.508
step 72/328, cls_loss = 0.417
step 108/328, cls_loss = 0.358
step 144/328, cls_loss = 0.468
step 180/328, cls_loss = 0.437
step 216/328, cls_loss = 0.691
step 252/328, cls_loss = 0.333
step 288/328, cls_loss = 0.549
step 324/328, cls_loss = 0.627
step 0/328, cls_loss = 0.458
checkpoint has been created!


100%|██████████| 36/36 [00:09<00:00,  3.71it/s]


validate has been finished!
val_acc = 0.650
step 36/328, cls_loss = 0.538
step 72/328, cls_loss = 0.486
step 108/328, cls_loss = 0.503
step 144/328, cls_loss = 0.395
step 180/328, cls_loss = 0.370
step 216/328, cls_loss = 0.552
step 252/328, cls_loss = 0.513
step 288/328, cls_loss = 0.421
step 324/328, cls_loss = 0.409
step 0/328, cls_loss = 0.289
checkpoint has been created!


100%|██████████| 36/36 [00:09<00:00,  3.72it/s]


validate has been finished!
val_acc = 0.652
step 36/328, cls_loss = 0.411
step 72/328, cls_loss = 0.351
step 108/328, cls_loss = 0.360
step 144/328, cls_loss = 0.439
step 180/328, cls_loss = 0.411
step 216/328, cls_loss = 0.563
step 252/328, cls_loss = 0.474
step 288/328, cls_loss = 0.338
step 324/328, cls_loss = 0.298


In [18]:
!mkdir -r /kaggle/working/model
!cp /kaggle/working/checkpoint2/epoch_0_iter_0.pth /kaggle/working/model
!cp /kaggle/working/checkpoint2/epoch_2_iter_0.pth /kaggle/working/model
!cp /kaggle/working/checkpoint2/epoch_9_iter_0.pth /kaggle/working/model

mkdir: invalid option -- 'r'
Try 'mkdir --help' for more information.


In [19]:
!zip -r model.zip /kaggle/working/model

import kagglehub

# Replace with path to directory containing model files.
LOCAL_MODEL_DIR = '/kaggle/working/model.zip'

MODEL_SLUG = 'faceAlexnet_pretrain' # Replace with model slug.

# Learn more about naming model variations at
# https://www.kaggle.com/docs/models#name-model.
VARIATION_SLUG = 'default' # Replace with variation slug.

kagglehub.model_upload(
  handle = f"poongln/{MODEL_SLUG}/keras/{VARIATION_SLUG}",
  local_model_dir = LOCAL_MODEL_DIR,
  version_notes = 'Update 2024-12-20')

  adding: kaggle/working/model/ (stored 0%)
  adding: kaggle/working/model/epoch_0_iter_0.pth (deflated 8%)
  adding: kaggle/working/model/epoch_2_iter_0.pth (deflated 8%)
  adding: kaggle/working/model/epoch_9_iter_0.pth (deflated 7%)
Uploading Model https://www.kaggle.com/models/poongln/faceAlexnet_pretrain/keras/default ...
Model 'faceAlexnet_pretrain' does not exist or access is forbidden for user 'poongln'. Creating or handling Model...
Model 'faceAlexnet_pretrain' Created.
Starting upload for file /kaggle/working/model.zip


Uploading: 100%|██████████| 631M/631M [00:22<00:00, 28.5MB/s] 

Upload successful: /kaggle/working/model.zip (602MB)


Your model instance has been created.
Files are being processed...
See at: https://www.kaggle.com/models/poongln/faceAlexnet_pretrain/keras/default
